In [63]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [64]:
import matplotlib.pyplot as plt

# Change the plot size.
plt.rcParams['figure.figsize'] = [18.0, 12.0]

In [65]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [66]:
# import my Finance Analysis Tools (fat)
import finance as fat

In [219]:
TICKER = 'IAU'
PATH = f'data/{TICKER}-test'
FILE = 'IAU-predictions-20180223-003722.csv'

In [237]:
data = fat.get_price_data(TICKER)

Loaded data for IAU: 2005-01-28 to 2018-02-21.


In [238]:
data['Next Day Adj Close'] = data['Adj Close'].shift(-1).copy()
data['Next Day Gain'] = data['Next Day Adj Close'] - data['Adj Close']

In [239]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,Next Day Adj Close,Next Day Gain
Date,,,,,,,,
2005-01-28,4.273,4.273,4.246,4.269,4.269,5777000,4.227,-0.042
2005-01-31,4.223,4.231,4.199,4.227,4.227,1519000,4.212,-0.015
2005-02-01,4.211,4.214,4.197,4.212,4.212,695000,4.223,0.011
2005-02-02,4.223,4.223,4.204,4.223,4.223,2993000,4.172,-0.051
2005-02-03,4.158,4.174,4.151,4.172,4.172,1068000,4.150,-0.022


In [240]:
predict = pd.read_csv(f'{PATH}/{FILE}')
predict = predict.set_index('Date')

In [241]:
predict.head()

,Month,Week,Day,Dayofweek,Dayofyear,Days since updated gold price usd,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,...,Volume SMA100,Volume SMA200,pct diff Volume SMA3,pct diff Volume SMA5,pct diff Volume SMA10,pct diff Volume SMA20,pct diff Volume SMA50,pct diff Volume SMA100,pct diff Volume SMA200,Will Close Higher Tomorrow
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-21,8,34,21,4,233,20.0,False,False,False,False,...,2549062.0,3641629.5,-15.692302,18.573483,42.078293,49.529427,41.621037,78.602950,25.018210,26.086124
2015-08-24,8,35,24,0,236,23.0,False,False,False,False,...,2604616.0,3638480.0,6.049517,48.946842,109.839340,132.049710,128.146760,192.999040,109.744180,48.456825
2015-08-25,8,35,25,1,237,24.0,False,False,False,False,...,2643510.0,3644522.5,-4.794828,-3.983591,43.462845,62.879505,65.800610,114.268150,55.416794,57.432760
2015-08-26,8,35,26,2,238,25.0,False,False,False,False,...,2644652.0,3642706.0,-46.942250,-52.561325,-25.427914,-18.604883,-17.126911,8.017993,-21.577530,143.165620
2015-08-27,8,35,27,3,239,26.0,False,False,False,False,...,2688441.0,3656939.5,18.409332,5.786842,32.002270,50.331715,58.115566,106.669960,51.935790,50.143867


In [242]:
data = data.join(predict['Will Close Higher Tomorrow'])

In [243]:
data = data.dropna()
data.head()

,Open,High,Low,Close,Adj Close,Volume,Next Day Adj Close,Next Day Gain,Will Close Higher Tomorrow
Date,,,,,,,,,
2015-08-21,11.18,11.23,11.11,11.20,11.20,4552700,11.14,-0.06,26.086124
2015-08-24,11.26,11.31,11.07,11.14,11.14,7631500,11.02,-0.12,48.456825
2015-08-25,11.09,11.10,10.96,11.02,11.02,5664200,10.87,-0.15,57.432760
2015-08-26,10.87,10.90,10.80,10.87,10.87,2856700,10.85,-0.02,143.165620
2015-08-27,10.83,10.90,10.80,10.85,10.85,5556200,10.95,0.10,50.143867


In [245]:
thresh100 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 100.0])
thresh100

,Open,High,Low,Close,Adj Close,Volume,Next Day Adj Close,Next Day Gain,Will Close Higher Tomorrow
Date,,,,,,,,,
2015-08-26,10.87,10.90,10.80,10.87,10.87,2856700,10.85,-0.02,143.165620
2015-12-17,10.18,10.19,10.12,10.15,10.15,10515000,10.29,0.14,102.566790
2015-12-31,10.24,10.27,10.22,10.23,10.23,5240700,10.38,0.15,108.538216
2016-02-22,11.65,11.73,11.64,11.65,11.65,4084700,11.84,0.19,116.741720
2016-03-24,11.78,11.81,11.74,11.75,11.75,26374100,11.78,0.03,104.844290
2016-06-16,12.65,12.69,12.34,12.36,12.36,14666900,12.52,0.16,122.531220
2016-08-24,12.81,12.82,12.75,12.77,12.77,4695300,12.75,-0.02,102.327415
2016-11-14,11.72,11.83,11.67,11.73,11.73,17358800,11.83,0.10,196.813980
2016-11-15,11.77,11.84,11.75,11.83,11.83,9865900,11.79,-0.04,143.487800


In [246]:
thresh100['Next Day Gain'].sum()

0.7300000000000022

In [252]:
thresh95 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 95.0])
thresh95

,Open,High,Low,Close,Adj Close,Volume,Next Day Adj Close,Next Day Gain,Will Close Higher Tomorrow
Date,,,,,,,,,
2015-08-26,10.87,10.90,10.80,10.87,10.87,2856700,10.85,-0.02,143.165620
2015-12-17,10.18,10.19,10.12,10.15,10.15,10515000,10.29,0.14,102.566790
2015-12-31,10.24,10.27,10.22,10.23,10.23,5240700,10.38,0.15,108.538216
2016-02-22,11.65,11.73,11.64,11.65,11.65,4084700,11.84,0.19,116.741720
2016-03-24,11.78,11.81,11.74,11.75,11.75,26374100,11.78,0.03,104.844290
2016-04-04,11.77,11.78,11.72,11.72,11.72,5664300,11.88,0.16,98.224625
2016-06-16,12.65,12.69,12.34,12.36,12.36,14666900,12.52,0.16,122.531220
2016-08-24,12.81,12.82,12.75,12.77,12.77,4695300,12.75,-0.02,102.327415
2016-11-14,11.72,11.83,11.67,11.73,11.73,17358800,11.83,0.10,196.813980


In [253]:
thresh95['Next Day Gain'].sum()

1.0100000000000016

In [247]:
thresh90 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 90.0])
thresh90

,Open,High,Low,Close,Adj Close,Volume,Next Day Adj Close,Next Day Gain,Will Close Higher Tomorrow
Date,,,,,,,,,
2015-08-26,10.87,10.90,10.80,10.87,10.87,2856700,10.85,-0.02,143.165620
2015-12-17,10.18,10.19,10.12,10.15,10.15,10515000,10.29,0.14,102.566790
2015-12-31,10.24,10.27,10.22,10.23,10.23,5240700,10.38,0.15,108.538216
2016-01-14,10.48,10.51,10.34,10.39,10.39,5323600,10.51,0.12,92.996185
2016-02-22,11.65,11.73,11.64,11.65,11.65,4084700,11.84,0.19,116.741720
2016-03-24,11.78,11.81,11.74,11.75,11.75,26374100,11.78,0.03,104.844290
2016-04-01,11.72,11.81,11.66,11.81,11.81,11475200,11.72,-0.09,91.605980
2016-04-04,11.77,11.78,11.72,11.72,11.72,5664300,11.88,0.16,98.224625
2016-05-26,11.87,11.87,11.75,11.76,11.76,4090400,11.68,-0.08,90.136110


In [248]:
thresh90['Next Day Gain'].sum()

0.9100000000000019

In [249]:
thresh80 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 80.0])
thresh80

,Open,High,Low,Close,Adj Close,Volume,Next Day Adj Close,Next Day Gain,Will Close Higher Tomorrow
Date,,,,,,,,,
2015-08-26,10.87,10.90,10.80,10.87,10.87,2856700,10.85,-0.02,143.165620
2015-11-05,10.69,10.71,10.65,10.65,10.65,5383600,10.50,-0.15,85.250244
2015-12-17,10.18,10.19,10.12,10.15,10.15,10515000,10.29,0.14,102.566790
2015-12-31,10.24,10.27,10.22,10.23,10.23,5240700,10.38,0.15,108.538216
2016-01-14,10.48,10.51,10.34,10.39,10.39,5323600,10.51,0.12,92.996185
2016-02-22,11.65,11.73,11.64,11.65,11.65,4084700,11.84,0.19,116.741720
2016-03-04,12.23,12.37,12.14,12.17,12.17,22198400,12.23,0.06,82.814060
2016-03-24,11.78,11.81,11.74,11.75,11.75,26374100,11.78,0.03,104.844290
2016-04-01,11.72,11.81,11.66,11.81,11.81,11475200,11.72,-0.09,91.605980


In [251]:
thresh80['Next Day Gain'].sum()

0.8400000000000034

In [254]:
thresh70 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 70.0])
thresh70['Next Day Gain'].sum()

0.09000000000000696

In [255]:
thresh60 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 60.0])
thresh60['Next Day Gain'].sum()

7.105427357601002e-15

In [256]:
thresh50 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 50.0])
thresh50['Next Day Gain'].sum()

0.7800000000000029

In [257]:
thresh40 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 40.0])
thresh40['Next Day Gain'].sum()

-0.05999999999999872

In [258]:
thresh30 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 30.0])
thresh30['Next Day Gain'].sum()

0.909999999999993

In [259]:
thresh20 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 20.0])
thresh20['Next Day Gain'].sum()

0.5699999999999914

In [260]:
thresh10 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 10.0])
thresh10['Next Day Gain'].sum()

1.5099999999999998

In [262]:
thresh5 = pd.DataFrame(data.loc[data['Will Close Higher Tomorrow'] >= 5.0])
thresh5['Next Day Gain'].sum()

1.5600000000000005